In [1]:
import numpy as np
from PIL import Image, ImageOps, ImageFilter, ImageDraw
from sklearn.metrics import mean_squared_error
import glob
import time


In [22]:
import numpy as np
from PIL import Image


def default_phi(x, mode=1, width=5):
    
    if mode == 1:
        phi = -1 * np.ones([x.shape[0], x.shape[1]])
        phi[int(x.shape[0] / 2) - width:int(x.shape[0] / 2) + width,
            int(x.shape[1] / 2) - width:int(x.shape[1] / 2) + width] = 1
        
    elif mode == 2:
        phi = 1. * np.ones([x.shape[0], x.shape[1]])
        phi[5:x.shape[0] - width, width:x.shape[1] - width] = -1.
        
    return phi

def sobel_filter(image):
    
    Kx = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]])
    Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
    
    Ix = convolve(image, Kx)
    Iy = convolve(image, Ky)
    
    G = np.sqrt(Ix**2 + Iy**2)
    
    return Ix, Iy, G

def gaussian_blur(image, kernel_size=3, sigma=1):
    
    kernel = np.zeros((kernel_size, kernel_size))
    
    for x in range(kernel_size):
        for y in range(kernel_size):
            kernel[x, y] = (1 / (2 * np.pi * sigma ** 2)) * np.exp(-((x - (kernel_size - 1) / 2) ** 2 + (y - (kernel_size - 1) / 2) ** 2) / (2 * sigma ** 2))
    
    kernel /= np.sum(kernel)
    
    return convolve(image, kernel)

def convolve(image, kernel):
    
    image_height, image_width = image.shape
    kernel_height, kernel_width = kernel.shape
    
    pad_height = kernel_height // 2
    pad_width = kernel_width // 2
    
    padded_image = np.pad(image, ((pad_height, pad_height), (pad_width, pad_width)), mode='constant')
    
    output = np.zeros_like(image)
    
    for i in range(image_height):
        for j in range(image_width):
            region = padded_image[i:i+kernel_height, j:j+kernel_width]
            output[i, j] = np.sum(region * kernel)
            
    return output

def custom_gradient(arr, axis=None):
    
    arr = np.asarray(arr, dtype=float)
    
    if axis is None:
        return [custom_gradient(arr, axis=i) for i in range(arr.ndim)]

    # Get the shape and the slices for computation
    dim = arr.shape[axis]
    slice_prefix = (slice(None),) * axis
    slice_center = slice_prefix + (slice(1, -1),)
    slice_front = slice_prefix + (slice(0, 1),)
    slice_back = slice_prefix + (slice(-1, None),)

    # Compute gradient
    grad = np.empty_like(arr, dtype=float)
    grad[slice_center] = (arr[slice_prefix + (slice(2, None),)] - arr[slice_prefix + (slice(None, -2),)]) / 2.0
    grad[slice_front] = arr[slice_prefix + (slice(1, 2),)] - arr[slice_front]
    grad[slice_back] = arr[slice_back] - arr[slice_prefix + (slice(-2, -1),)]

    return grad




def binary_opening(image, structure):
    
        eroded = binary_erosion(image, structure=structure)
        opened = binary_dilation(eroded, structure=structure)
        
        return opened

def binary_closing(image, structure):
    
        dilated = binary_dilation(image, structure=structure)
        closed = binary_erosion(dilated, structure=structure)
        
        return closed

def binary_erosion(image, structure):
    
        height, width = image.shape
        structure_height, structure_width = structure.shape
        
        pad_height = structure_height // 2
        pad_width = structure_width // 2
        
        padded_image = np.zeros((height + 2 * pad_height, width + 2 * pad_width))
        padded_image[pad_height:pad_height + height, pad_width:pad_width + width] = image
        
        output = np.zeros_like(image)
        
        for i in range(height):
            for j in range(width):
                region = padded_image[i:i+structure_height, j:j+structure_width]
                output[i, j] = np.min(region[structure.astype(bool)])
                
        return output

def binary_dilation(image, structure):
    
        height, width = image.shape
        structure_height, structure_width = structure.shape
        
        pad_height = structure_height // 2
        pad_width = structure_width // 2
        
        padded_image = np.zeros((height + 2 * pad_height, width + 2 * pad_width))
        padded_image[pad_height:pad_height + height, pad_width:pad_width + width] = image
        
        output = np.zeros_like(image)
        
        for i in range(height):
            for j in range(width):
                region = padded_image[i:i+structure_height, j:j+structure_width]
                output[i, j] = np.max(region[structure.astype(bool)])
                
        return output




def remove_noise(binary_image, structure=None):
    
    if structure is None:
        structure = np.ones((3, 3))
        
    cleaned = binary_closing(binary_opening(binary_image, structure=structure), structure=structure)
    
    return cleaned


def mean_squared_error(y_true, y_pred):
    
    error = np.mean((y_true - y_pred) ** 2)
    
    return error


def lss(img, dt=3.5, freq=10, rad=3, b=1):
    
    # print("Original image mode:", img.mode)
    
    img = img.convert('L')
    
    # print("Converted to grayscale:", img.mode)
    
    img = np.array(img)
    img = img - np.mean(img)

    img = gaussian_blur(img)  # Apply Gaussian blur
    
    # print("Gaussian blur applied.")


    # new
    if b == 2:
        dx, dy, Du = sobel_filter(img)
        v = 1. / (1. + 0.5 * Du)   
    
    
    
    # old
    if b == 1:
        dx, dy = np.gradient(img)
        Du = np.sqrt(dx**2 + dy**2)
        v = 1. / (1. + Du) 
    
    
    # print("Sobel edge strength calculated.")

    u = default_phi(img, mode=2)
    data = [Du, u]
    u_old = u
    
    niter = 0
    MSE_OLD = 1e+03
    change = 1e+03

    while change > 1e-15:
        
        niter += 1
        
        dx = custom_gradient(u, axis=1)
        dy = custom_gradient(u, axis=0)
        
        Du = np.sqrt(dx ** 2 + dy ** 2)  # internal energy
        
        u += dt * v * Du
        u = np.where(u < 0, -1., 1.)

        MSE = mean_squared_error(u_old, u)
        
        u_old = u
        
        change = abs(MSE - MSE_OLD)
        
        MSE_OLD = MSE
        
        if niter % freq == 0:
            data.append(u)

    u = np.where(u < 0, 1., 0.)

    # Remove noise from binary output
    u = remove_noise(u)
    data.append(u)
    
    return data, niter








In [23]:
def plot_boundary(img, segment):
  rad = 3
  img = img.convert("RGB")
  edge = Image.fromarray(segment).convert("L").filter(ImageFilter.GaussianBlur(radius = rad)).filter(ImageFilter.FIND_EDGES)
  for x in range(rad,img.size[0]-rad):
      for y in range(rad,img.size[1]-rad):
        if edge.getpixel((x,y)) != 0:
            for i in [-1,0,1]:
                for j in [-1,0,1]:
                    img.putpixel( (x+i,y+j), (255,0,255) )

  return img

#Saves the boundary evolution as a gif file
def evolution_gif(img, data, file_name = 'dummy.gif', duration = 400, loop = 2):
  images = []
  for i in range(2,len(data)):
    tmp =plot_boundary(img, data[i])
    images.append(tmp)

  images[0].save(file_name, save_all=True, append_images=images[1:], optimize=False, duration=duration, loop=loop)



In [62]:
import numpy as np

def dice_coefficient(image1, image2):
    intersection = np.logical_and(image1, image2)
    dice = (2.0 * intersection.sum()) / (image1.sum() + image2.sum())
    return dice

def jaccard_index(image1, image2):
    intersection = np.logical_and(image1, image2)
    union = np.logical_or(image1, image2)
    jaccard = intersection.sum() / union.sum()
    return jaccard

def precision(image1, image2):
        true_positive = np.logical_and(image1, image2).sum()
        predicted_positive = image1.sum()
        if predicted_positive == 0:
                predicted_positive = 1
        precision = true_positive / predicted_positive
        return precision

def recall(image1, image2):
        true_positive = np.logical_and(image1, image2).sum()
        actual_positive = image2.sum()
        recall = true_positive / actual_positive
        return recall

def f1_score(image1, image2):
        prec = precision(image1, image2)
        rec = recall(image1, image2)
        f1 = 2 * (prec * rec) / (prec + rec)
        return f1

def accuracy(image1, image2):
    image1 = image1.astype(bool)
    image2 = image2.astype(bool)
    true_positive = np.logical_and(image1, image2).sum()
    true_negative = np.logical_and(~image1, ~image2).sum()
    total_pixels = image1.size
    acc = (true_positive + true_negative) / total_pixels
    return acc

    # Example usage:
    
    



In [63]:
#state of the art

import torch
import torchvision
import torch
from PIL import Image
import torchvision.transforms as T


def saa(image):


    # Download the pre-trained model and weights
    model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=True)

    # Save the model weights to a file
    # torch.save(model.state_dict(), 'segmentation_model_weights.pth')

    # Load the pre-trained model and weights
    model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=True)
    model.eval()

    # Load the image
    # image_path = 'tmpz9so4ise.PNG'
    # image = Image.open(image_path)

    # Preprocess the image
    transform = T.Compose([
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    input_image = transform(image).unsqueeze(0)

    # Run the image through the model
    with torch.no_grad():
        output = model(input_image)['out'][0]
        output = torch.argmax(output, dim=0).byte().numpy()

    # Save the binary segmented map
    output_image = Image.fromarray(output)
    # output_image.save('segmented_map.png')
    binary_output_image = output_image.point(lambda x: 0 if x == 0 else 1, mode='L')
    
    
    return binary_output_image


In [61]:
import os
from PIL import Image
from PIL import ImageDraw
import os
from PIL import Image
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt

# Directory containing the images and annotations
image_dir = 'VOCdevkit/VOC2012/JPEGImages'
annotation_dir = 'VOCdevkit/VOC2012/Annotations'

# List to store image paths and corresponding annotations
image_annotations = []

# Iterate over the image directory
for filename in os.listdir(image_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(image_dir, filename)
        annotation_path = os.path.join(annotation_dir, filename.replace('.jpg', '.xml').replace('.png', '.xml'))
        
        # Check if the annotation file exists
        if os.path.exists(annotation_path):
            
            ground_truth_path = os.path.join('VOCdevkit/VOC2012/SegmentationObject', filename.replace('.jpg', '.png'))
            if os.path.exists(ground_truth_path):
            
                image_annotations.append((image_path, annotation_path))
            

# Function to parse the annotation XML file
def parse_annotation(annotation_path):
    tree = ET.parse(annotation_path)
    root = tree.getroot()
    
    # Extract the required information from the XML tree
    # Replace this code with your own logic to extract the desired information
    
    # Example: Extracting bounding box coordinates
    bbox = root.find('object').find('bndbox')
    xmin = int(bbox.find('xmin').text)
    ymin = int(bbox.find('ymin').text)
    xmax = int(bbox.find('xmax').text)
    ymax = int(bbox.find('ymax').text)
    
    return xmin, ymin, xmax, ymax





    

# Iterate over the image annotations
# print(image_annotations)



# anots = [79,22]
# [154:155]

image_annotations = image_annotations[0:50]
# image_annotations = image_annotations[0:17] + image_annotations[18:36]# + image_annotations[37:50]
# image_annotations = image_annotations[150:155]
# image_annotations = image_annotations[21:22]


sum_jaccard_index = 0
sum_f1_score = 0
sum_accuracy = 0

count = 0

# sum_dice_coefficient = 0
# sum_precision = 0
# sum_recall = 0



parameter = 3.5


for image_path, annotation_path in image_annotations:
    # Load the image
    image = Image.open(image_path)
    # plt.imshow(image)
    # plt.show()
    
    # Parse the annotation XML file
    xmin, ymin, xmax, ymax = parse_annotation(annotation_path)
    
    # Process the image and annotation as required
    # Replace this code with your own logic to process the image and annotation
    
    
    
    # Example: Display the image and bounding box
    image_with_bbox = image.copy()
    draw = ImageDraw.Draw(image_with_bbox)
    draw.rectangle([(xmin, ymin), (xmax, ymax)], outline='red')
    # image_with_bbox.show()
    
    
    image.size[0]


    
    # img = image[xmin:xmax, ymin:ymax]
    img = np.array(image)[max(0,ymin-10):min(image.size[1],ymax+10), max(0,xmin-10):min(image.size[0],xmax+10)]
    img = Image.fromarray(img)
    
    
    
    
    # level set
    data, niter = lss(img, dt = parameter, freq = 10, b=1) # b=1 for old, b=2 for new
    
    
    # state of the art
    # data = []
    # data.append(saa(img))

    
    # display(ImageOps.autocontrast(Image.fromarray(data[-1]).convert('L')))  
    # display(plot_boundary(img, data[-1]))
    
    
    
    
    # timestr = time.strftime("%Y%m%d-%H%M%S")
    # evolution_gif(img, data, file_name = timestr+'.gif', duration=800, loop=2)
    



    
    # Directory containing the ground truth images
    ground_truth_dir = 'VOCdevkit/VOC2012/SegmentationObject'

    # Iterate over the image annotations
    # for image_path, annotation_path in image_annotations:
        # Get the filename of the image
    filename = os.path.basename(image_path)
        
        # Construct the path to the ground truth image
    ground_truth_path = os.path.join(ground_truth_dir, filename.replace('.jpg', '.png'))
        
        # Load the ground truth image
    ground_truth_img = Image.open(ground_truth_path)
        
        # Display the ground truth image
    # plt.imshow(ground_truth_img)
    # plt.title("Ground Truth Image")
    # plt.axis('off')
    # plt.show()
    
    
    gt_img = np.array(ground_truth_img)[max(0,ymin-10):min(ground_truth_img.size[1],ymax+10), max(0,xmin-10):min(ground_truth_img.size[0],xmax+10)]
    gt_img = Image.fromarray(gt_img)
    
    
    ground_truth_binary = gt_img.point(lambda x: 0 if x == 0 else 1, mode='1')
    # plt.imshow(ground_truth_binary, cmap='gray')
    # # plt.title("Binary Ground Truth Image")
    # plt.axis('off')
    # plt.show()
    


    # Convert PIL Image to NumPy array
    ground_truth_binary_np = np.array(ground_truth_binary)
    data_last_np = np.array(data[-1])
    
    
    
    sum_jaccard_index += jaccard_index(data_last_np, ground_truth_binary_np)
    sum_f1_score += f1_score(data_last_np, ground_truth_binary_np)
    sum_accuracy += accuracy(data_last_np, ground_truth_binary_np)
    
    count += 1
    
    
    # if jaccard_index(data_last_np, ground_truth_binary_np) > 0.0:
        
        # display(ImageOps.autocontrast(Image.fromarray(data[-1]).convert('L')))  
        # display(plot_boundary(img, data[-1]))
        
        
        


        # fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        # for ax in axs:
        #     ax.imshow(data[-1], cmap='gray')
        #     ax.axis('off')
        #     ax.spines['top'].set_color('black')
        #     ax.spines['bottom'].set_color('black')
        #     ax.spines['left'].set_color('black')
        #     ax.spines['right'].set_color('black')


        # axs[0].imshow(data[-1], cmap='summer')
        # axs[0].axis('off')
        
        # axs[1].imshow(ground_truth_binary_np, cmap='summer')
        # axs[1].axis('off')

        # axs[2].imshow(img)
        # axs[2].axis('off')
        

        # plt.show()
        
        
        
        # plt.imshow(data[-1], cmap='summer')
        # plt.axis('off')
        # plt.show()
        # plt.imshow(img)
        # plt.axis('off')
        # plt.show()
        # plt.imshow(ground_truth_binary_np, cmap='summer')
        # plt.axis('off')
        # plt.show()
        
        # print("")
        
        

    # # Now pass these NumPy arrays to your functions
    # print("jaccard_index: ", jaccard_index(data_last_np, ground_truth_binary_np))
    # # print("dice_coefficient: ", dice_coefficient(data_last_np, ground_truth_binary_np))
    # # print("Precision: ", precision(data_last_np, ground_truth_binary_np))
    # # print("Recall: ", recall(data_last_np, ground_truth_binary_np))
    # print("F1 Score: ", f1_score(data_last_np, ground_truth_binary_np))
    # print("Accuracy: ", accuracy(data_last_np, ground_truth_binary_np))
    
    print(count)
    
    
# results.append((parameter, (sum_jaccard_index/count, sum_f1_score/count, sum_accuracy/count)))
    
    
    
    
print(" ------------------------ ")
print("overall jaccard_index: ", sum_jaccard_index/count)
# print("dice_coefficient: ", dice_coefficient(data_last_np, ground_truth_binary_np))
# print("Precision: ", precision(data_last_np, ground_truth_binary_np))
# print("Recall: ", recall(data_last_np, ground_truth_binary_np))
print("overall F1 Score: ", sum_f1_score/count)
print("overall Accuracy: ", sum_accuracy/count)
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
 ------------------------ 
overall jaccard_index:  0.7476681488277658
overall F1 Score:  0.8371054383359914
overall Accuracy:  0.8500879899863231


In [25]:
results

results = sorted(results)

import matplotlib.pyplot as plt

parameters = [result[0] for result in results]
jaccard_index = [result[1][0] for result in results]
f1_score = [result[1][1] for result in results]
accuracy = [result[1][2] for result in results]

plt.figure(figsize=(8, 6))
plt.plot(parameters, jaccard_index, marker='o', label='Jaccard Index')
plt.plot(parameters, f1_score, marker='o', label='F1 Score')
plt.plot(parameters, accuracy, marker='o', label='Accuracy')
plt.xlabel('dt')
plt.ylabel('Score')
plt.title('Evaluation Results')
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'results' is not defined